In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import missingno as msno  # missingno库提供了一个很好的可视化缺失数据的方式
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, PowerTransformer, QuantileTransformer
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt

from datetime import datetime
import statsmodels.api as sm


In [ ]:
df = pd.read_csv('merged_financial_stock_macro_data_with_better_features.csv', index_col=0)

In [ ]:
# df = final_data.copy()

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# 确保数据按 TICKER 和 date 排序
df = df.sort_values(['TICKER', 'date'])

In [ ]:
df.reset_index(inplace = True)

df.set_index(['date'], inplace=True)

def dropIndex(dataset):
    try:
        dataset = dataset.drop(columns = ['level_0', 'index'])
    except Exception as e:
                print(f"Issue : {e}")
    return dataset
df = dropIndex(df)

In [ ]:
df.index

In [ ]:
# existing_vars = [
#     'market_cap',  # ME (Market Equity)
#     'bm',          # Book-to-Market
#     'pe_op_basic', # E2P (inverse)
#     'pe_op_dil',   # Another version of E2P (inverse)
#     'npm',         # Similar to PM (Profit Margin)
#     'gpm',         # Similar to PCM (Price-to-Cost Margin)
#     'roa',         # Return on Assets
#     'roe',         # Return on Equity
#     'ps',          # Price to Sales (inverse of S2P)
#     'debt_assets', # Similar to Lev (Leverage)
#     'at_turn',     # Similar to ATO (Asset Turnover)
#     'accrual',     # Might correspond to OA (Operating Accruals)
#     'divyield',    # Dividend Yield
# ]
benchmark = ['Size','Momentum','bm']

Green et al. (2017)的94个指标部分
Welch and Goyal (2008) 宏观指标：Treasury-bill rate (tbl), term spread (tms), default spread (dfy)

市场相关指标:


ME (Market Equity): 公司层面的总市值
LME: 上个月月末的总市值
LTurnover: 上月总交易量与上月月末总市值的比率
Spread: 上月日均买卖价差
Beta: 根据Frazzini和Pedersen (2014)的定义计算的贝塔值
Idio vol (Idiosyncratic Volatility): 基于Ang等(2006)方法计算的特异波动率



历史回报指标:


r2−1: 上个月的滞后回报
r12−2: 过去12个月(不包括最后一个月)的回报
r12−7: 中期动量(过去12个月到过去7个月的回报)
r36−13: 长期回报(3年前到去年的回报)


财务报表指标:


BE (Book Equity): 账面权益
BM (Book-to-Market): 账面市值比
AT: 总资产
ATO: 销售额除以净营运资产
C (Cash): 现金和短期投资占总资产比例
CTO: 资本周转率
D2A: 折旧和摊销占总资产比例
DPI2A: 固定资产变动
E2P: 市盈率的倒数
FC2Y: 费用销售比(广告、研发、销售管理费用总和除以销售额)
Free CF: 自由现金流
Investment: 总资产增长率
Lev: 杠杆率(总债务/(总债务+股东权益))
NOA: 净营运资产占滞后总资产比例
OA: 营运应计项目(根据Sloan (1996)定义)
OL: 营运杠杆(销货成本和销售管理费用除以总资产)
PCM: 价格成本利润率((销售额-销货成本)/销售额)
PM: 营业利润率(折旧后营业收入/净销售额)
Prof: 毛利润率(毛利润/权益账面价值)
Q: 托宾Q值
Rel to high: 接近52周最高价的程度
RNA: 净营运资产回报率
ROA: 资产回报率(税前利润/总资产)
ROE: 权益回报率(税前利润/滞后权益账面价值)
S2P: 销售额市值比
SGA2S: 销售管理费用率(销售管理费用/净销售额)
SUV: 标准化非预期交易量

根据Gu 2020年的论文，我们可以将特征分为以下几类：

基准特征（3个）：

规模（Size）
账面市值比（Book-to-Market ratio）
动量（Momentum）


股票水平预测器（主要分为三类）：
a. 价格趋势变量：

股票动量
行业动量
短期反转
b. 流动性变量：
市值
美元交易量
买卖价差
c. 波动性相关：
回报波动率
特质波动率
市场beta
beta平方


宏观经济预测器（8个）：

股息-价格比（Dividend-Price Ratio, dp）
收益-价格比（Earnings-Price Ratio, ep）
账面市值比（Book-to-Market Ratio, bm）
净股票发行（Net Equity Expansion, ntis）
国库券利率（Treasury-bill Rate, tbl）
期限利差（Term Spread, tms）
违约利差（Default Spread, dfy）
股票方差（Stock Variance, svar）


94个预测特征（基于Green et al. (2017)）

处理步骤：

基准特征和主要股票水平预测器：
这些特征大多可以从CRSP数据中直接计算或提取。
宏观经济预测器：
这些需要额外的宏观经济数据源，可能需要从Federal Reserve Economic Data (FRED)或其他经济数据库获取。
94个预测特征：
这些特征基于Green et al. (2017)，需要使用财务报表数据（如Compustat）结合市场数据来计算。
数据对齐：

月度特征：使用t月末的数据预测t+1月的回报
季度特征：使用至少4个月前的数据
年度特征：使用至少6个月前的数据


处理缺失值：
对于缺失的特征，使用每个月每只股票的横截面中位数进行填充

根据您提供的信息，我将帮您整理需要的变量、已有的变量和缺失的变量。

1. 需要的变量（基于您的要求）：

a. 基准特征：
   - Size (已有)
   - Book-to-Market ratio (已有，对应 'bm')
   - Momentum (已有)

b. 股票水平预测器：
   - 股票动量 (已有，对应 'Momentum')
   - 行业动量 (缺失)
   - 短期反转 (已有，对应 'ShortTermReversal')
   - 市值 (已有，对应 'ME' 或 'market_cap')
   - 美元交易量 (已有，对应 'DollarVolume')
   - 买卖价差 (已有，对应 'spread')
   - 回报波动率 (可以从 'RET' 计算)
   - 特质波动率 (已有，对应 'Idio_vol')
   - 市场beta (已有，对应 'Beta')
   - beta平方 (已有，对应 'BetaSquared')

c. 宏观经济预测器：
   - 股息-价格比 (dp) (已有，对应 'divyield')
   - 收益-价格比 (ep) (可以从 'pe_op_basic' 或 'pe_op_dil' 的倒数得到)
   - 账面市值比 (bm) (已有)
   - 净股票发行 (ntis) (缺失)
   - 国库券利率 (tbl) (可能对应 'TB3MS'，需确认)
   - 期限利差 (tms) (可能对应 'T10Y2Y'，需确认)
   - 违约利差 (dfy) (可能对应 'BAA10Y'，需确认)
   - 股票方差 (svar) (可以从 'RET' 计算)

2. 已有的重要变量（除上述外）：
   - 'roa', 'roe', 'roce' (盈利能力指标)
   - 'debt_assets', 'debt_capital', 'de_ratio' (杠杆指标)
   - 'cash_ratio', 'quick_ratio', 'curr_ratio' (流动性指标)
   - 'at_turn', 'inv_turn' (效率指标)
   - 'accrual' (质量指标)
   - 'PRC', 'VOL', 'SHROUT' (价格、交易量、股数)
   - 'vwretd', 'ewretd', 'sprtrn' (市场回报指标)

3. 缺失的重要变量：
   - 行业动量
   - 净股票发行 (ntis)

4. 可能不需要的变量（这取决于您的具体研究目的，但这些变量可能与其他变量重复或不太常用）：
   - 'CAPEI' (如果有 'pe_op_basic' 或 'pe_op_dil')
   - 'evm' (如果已有 'bm')
   - 'pe_exi', 'pe_inc' (如果已有 'pe_op_basic' 或 'pe_op_dil')
   - 'ptpm' (如果已有 'npm')
   - 'pcf' (如果已有 'ps' 和 'cfm')
   - 'aftret_eq', 'aftret_invcapx', 'aftret_equity' (如果已有 'roe' 和 'roce')
   - 'pretret_noa', 'pretret_earnat' (如果已有其他盈利能力指标)
   - 'equity_invcap', 'debt_invcap', 'totdebt_invcap' (如果已有 'debt_capital' 和 'de_ratio')
   - 'intcov', 'intcov_ratio' (可能重复)
   - 'sale_invcap', 'sale_equity', 'sale_nwc' (如果已有 'ps' 和其他效率指标)
   - 'int_debt', 'int_totdebt' (如果已有其他债务指标)
   - 'invt_act', 'rect_act' (如果已有 'curr_ratio' 和 'quick_ratio')
   - 'short_debt', 'curr_debt', 'lt_debt' (如果已有 'debt_assets' 和 'debt_capital')
   - 'ptb' (如果已有 'bm')
   - 'PEG_trailing' (如果已有 'pe_op_basic' 或 'pe_op_dil' 和增长率指标)

对于缺失的变量和需要计算的变量，您可能需要获取额外的数据或使用现有数据进行计算。例如，行业动量可能需要行业分类数据，而回报波动率和股票方差可以使用现有的回报数据计算。

In [ ]:
def calculate_variables(df):
    # 0 benchmark
    df['BM'] = df['bm']
    
    # ME (Market Equity)
    df['ME'] = df['market_cap']
    # Size (natural log of ME)
    df['Size'] = np.log(df['ME'])
    
    # Momentum (11-month return, 1 month ago)
    def calculate_momentum(group):
        return group.rolling(window=11).apply(lambda x: (1 + x).prod() - 1).shift(1)

    df['Momentum'] = df.groupby('TICKER')['RET'].transform(calculate_momentum)

    # 1 市场相关指标
    # 短期反转（过去1个月的回报）
    df['ShortTermReversal'] = df.groupby('TICKER')['RET'].shift(1)
    
    # # 计算市值
    # df['market_cap'] = df['PRC'].abs() * df['SHROUT']
    
    # # LME (Lagged Market Equity) 上个月月末的总市值
    # df['LME'] = df.groupby('TICKER')['market_cap'].shift(1)
    
    # # LTurnover 上月总交易量与上月月末总市值的比率
    # df['LTurnover'] = df['VOL'] / df['LME']
    
    # # 美元交易量
    # df['DollarVolume'] = df['PRC'].abs() * df['VOL']

    # # Spread 上月日均买卖价差
    # df['spread'] = (df['ASKHI'] - df['BIDLO']) / ((df['ASKHI'] + df['BIDLO']) / 2)
    
    # # 计算20天滚动标准差
    # df['Volatility'] = df.groupby('TICKER')['RET'].transform(lambda x: x.rolling(window=20, min_periods=1).std())
    # # 处理可能的 NaN 值
    # df['Volatility'] = df['Volatility'].fillna(0)

    # 重置索引
    # df_reset = df.reset_index()
    
    # # Beta (simplified version) 用滚动算
    # def calculate_beta(stock_returns, market_returns, window=52):
    #     return stock_returns.rolling(window=window).apply(lambda x: np.cov(x, market_returns[-window:])[0,1] / np.var(market_returns[-window:]))
    
    # df_reset['Beta'] = df.groupby('TICKER')['RET'].apply(lambda x: calculate_beta(x, df['sprtrn']))

    # # 计算Beta平方
    # df_reset['BetaSquared'] = df_reset['Beta'] ** 2

    # df['Beta'] = df_reset['Beta']
    # df['BetaSquared'] = df_reset['BetaSquared']


    # 2 历史回报指标
    # # Historical returns
    # df['r2_1'] = df.groupby('TICKER')['RET'].shift(1)

    # # 计算 r12_2
    # df_reset['r12_2'] = df_reset.groupby('TICKER')['RET'].rolling(window=11).sum().shift(1).reset_index(0, drop=True)
    # df_reset['r12_7'] = df_reset.groupby('TICKER')['RET'].rolling(window=6).sum().shift(6).reset_index(0, drop=True)
    # df_reset['r36_13'] = df_reset.groupby('TICKER')['RET'].rolling(window=24).sum().shift(12).reset_index(0, drop=True)

    # # 设置回原来的索引
    # df_reset.set_index('index', inplace=True)
    
    # # 将结果赋值给原始 DataFrame
    # df['r12_2'] = df_reset['r12_2']
    # df['r12_7'] = df_reset['r12_7']
    # df['r36_13'] = df_reset['r36_13']

    # df['RET_lag_1_mon'] = df.groupby('TICKER')['RET'].shift(1)

    # 3 财务报表指标
    
    # # 重置索引
    # df_reset = df.reset_index()
    
    # # 确保 'DATE' 列是 datetime 类型
    # df_reset['date'] = pd.to_datetime(df_reset['date'])
    
    # # 设置 'DATE' 列为索引
    # df_reset.set_index('date', inplace=True)
    
    # # 确保索引是升序排列的
    # df_reset.sort_index(inplace=True)

    # # Rel to high 接近52周最高价的程度
    # df_reset['52_week_high'] = df_reset.groupby('TICKER')['PRC'].rolling(window=52).max().reset_index(0, drop=True)
    
    # # 计算 Rel_to_high
    # df_reset['Rel_to_high'] = df_reset['PRC'] / df_reset['52_week_high']
    
    # 设置回原来的索引
    # df_reset.set_index('index', inplace=True)
    # print(df_reset['52_week_high'].isnull().sum())
    
    # # 将结果赋值给原始 DataFrame
    # df['52_week_high'] = df_reset['52_week_high']
    # df['Rel_to_high'] = df_reset['Rel_to_high']
    
    # SUV (Standardized Unexplained Volume)
    # def calculate_suv(volume, window=20):
    #     log_volume = np.log(volume)
    #     mean_log_volume = log_volume.rolling(window=window).mean()
    #     std_log_volume = log_volume.rolling(window=window).std()
    #     return (log_volume - mean_log_volume) / std_log_volume
    
    # df['SUV'] = df.groupby('TICKER')['VOL'].apply(calculate_suv)
    
    return df

# Apply the function to your dataframe
df = calculate_variables(df)

In [ ]:
# 计算缺失值统计，转换为 DataFrame
null_summary = pd.DataFrame(df.isnull().sum(), columns=['MissingValueCount'])

# 使用 option_context 临时改变显示设置并直接显示 DataFrame
with pd.option_context('display.max_columns', None,  'display.max_colwidth', 20):
    display(null_summary)

null_summary.to_csv("missing_value_summary.csv")


In [ ]:
# 处理无穷大和NaN值
df = df.replace([np.inf, -np.inf], np.nan)

In [ ]:
def clean_and_fill_numeric(df, var):
    # 尝试将数据转换为数值型
    df[var] = pd.to_numeric(df[var], errors='coerce')    
    return df
    
# 对每个需要处理的变量应用此函数
variables_to_clean = ['Momentum', 'r12_2', 'r12_7', 'r36_13', 'sprtrn', 'RET', 'RET_lag_1_mon']  # 添加所有需要处理的变量
for var in variables_to_clean:
    df = clean_and_fill_numeric(df, var)

In [ ]:
def handle_missing_values(df):
    # 对历史回报指标使用前向填充
    historical_returns = ['Momentum', 'r12_2', 'r12_7', 'r36_13', 'ShortTermReversal', 'LME', 'LTurnover', 'r2_1', 'RET_lag_1_mon']
    df[historical_returns] = df.groupby('TICKER')[historical_returns].fillna(method='ffill')
    df[historical_returns] = df.groupby('TICKER')[historical_returns].fillna(method='bfill')
    df[historical_returns] = df[historical_returns].fillna(df[historical_returns].median())

    
    # 对交易相关指标使用中位数填充
    trading_indicators = ['DollarVolume', 'spread']
    df[trading_indicators] = df[trading_indicators].fillna(df[trading_indicators].median())
    
    # 对SUV使用0填充
    df['SUV'] = df['SUV'].fillna(0)
    
    return df

# 应用函数
df = handle_missing_values(df)

In [ ]:
df.dtypes

In [ ]:
print(df.head())  # 检查前几行数据
print(df.isnull().sum())  # 计算NaN的数量


In [ ]:
df.sort_values(['date']).head()

In [ ]:
# def clean_and_fill_numeric(df, var):
#     # 尝试将数据转换为数值型
#     df[var] = pd.to_numeric(df[var], errors='coerce')    
#     return df
    
# # 对每个需要处理的变量应用此函数
# variables_to_clean = ['Momentum', 'r12_2', 'r12_7', 'r36_13', 'sprtrn', 'RET', 'RET_lag_1_mon']  # 添加所有需要处理的变量
# for var in variables_to_clean:
#     df = clean_and_fill_numeric(df, var)
    
def handle_remaining_missing_values(df):
    # 处理 Momentum 和长期回报指标
    long_term_vars = ['Momentum', 'r12_2', 'r12_7', 'r36_13']
    for var in long_term_vars:
        df[var] = df.groupby('SICCD')[var].transform(lambda x: x.fillna(x.median()))
        df[var] = df[var].fillna(df[var].median())

    # 获取仍然缺失 Momentum 的公司
    missing_momentum_companies = df[df['Momentum'].isna()]['TICKER'].unique()
    # 使用行业平均值填充
    df['Momentum'] = df.groupby('SICCD')['Momentum'].transform(lambda x: x.fillna(x.mean()))
    # 用总体平均值填充
    df['Momentum'] = df['Momentum'].fillna(df['Momentum'].median())

    # 处理短期指标
    short_term_vars = ['ShortTermReversal', 'LME', 'LTurnover', 'r2_1']
    df[short_term_vars] = df.groupby('TICKER')[short_term_vars].fillna(method='ffill')
    df[short_term_vars] = df.groupby('TICKER')[short_term_vars].fillna(method='bfill')
    df[short_term_vars] = df[short_term_vars].fillna(df[short_term_vars].median())

    for var in short_term_vars:
        df[var] = df.groupby('SICCD')[var].transform(lambda x: x.fillna(x.median()))
    
    # 重新计算 Beta 和相关指标
    def calculate_beta(stock_returns, market_returns, window=52):
        return stock_returns.rolling(window=window).apply(lambda x: np.cov(x, market_returns[-window:])[0,1] / np.var(market_returns[-window:]))

    df['Beta'] = df.groupby('TICKER')['RET'].apply(lambda x: calculate_beta(x, df['sprtrn']))
    df['BetaSquared'] = df['Beta'] ** 2
    
    # 对Beta相关指标使用行业平均值填充
    beta_indicators = ['Beta', 'BetaSquared','idio_vol']
    for indicator in beta_indicators:
        df[indicator] = df.groupby('SICCD')[indicator].transform(lambda x: x.fillna(x.median()))
    
    # 如果仍有缺失值（例如，某些公司可能全部为NaN），使用整体中位数填充
    for indicator in beta_indicators:
        df[indicator] = df[indicator].fillna(df[indicator].median())
    
    # 用行业中位数
    # df[indicator] = df.groupby('SICCD')[indicator].transform(lambda x: x.fillna(x.mean()))
    # # 如果仍有缺失值，使用整体中位数填充
    # df[indicator] = df[indicator].fillna(df[indicator].median())

    df_temp = pd.DataFrame()    
    
    # 对52周高点相关指标使用3个月高点替代
    df_temp['3_month_high'] = df.groupby('TICKER')['PRC'].rolling(window=63).max().reset_index(0, drop=True)
    df_temp['Rel_to_3month_high'] = df['PRC'] / df_temp['3_month_high']
    
    # 使用 1 个月高点
    df_temp['1_month_high'] = df.groupby('TICKER')['PRC'].rolling(window=21).max().reset_index(0, drop=True)
    df_temp['Rel_to_1month_high'] = df['PRC'] / df_temp['1_month_high']
    
    # 处理可能的缺失值
    df['52_week_high'] = df.groupby('TICKER')['52_week_high'].fillna(method='ffill',limit = 3)
    df['Rel_to_high'] = df.groupby('TICKER')['Rel_to_high'].fillna(method='ffill',limit = 3)

    # 如果还有缺失值，使用3个月高点填充52周高点
    df['52_week_high'] = df['52_week_high'].fillna(df_temp['3_month_high'])
    df['Rel_to_high'] = df['Rel_to_high'].fillna(df_temp['Rel_to_3month_high'])

    # 如果还有缺失值，使用1个月高点填充52周高点
    df['52_week_high'] = df['52_week_high'].fillna(df_temp['1_month_high'])
    df['Rel_to_high'] = df['Rel_to_high'].fillna(df_temp['Rel_to_1month_high'])

    # 如果还有缺失值，使用当前价格填充52周高点，Rel_to_high填充为1
    df['52_week_high'] = df['52_week_high'].fillna(df['PRC'])
    df['Rel_to_high'] = df['Rel_to_high'].fillna(1)
    
    return df

# 应用函数
df = handle_remaining_missing_values(df)

In [ ]:
pd.DataFrame(df.columns).to_csv('dfcolumns.csv')

In [ ]:
df.columns

In [ ]:
df.head() 

In [ ]:
# df.reset_index(inplace = True)

# df.set_index(['date','TICKER'], inplace=True)

In [ ]:
# df['date'].head()

In [ ]:
# import pandas as pd
# import numpy as np
# import statsmodels.api as sm

# def calculate_rolling_beta_and_idio_vol(df, window=60):
#     # 确保 df 是按日期和股票代码排序的
#     df = df.sort_values(['date', 'TICKER'])
    
#     # 计算市场收益率
#     df['market_returns'] = df.groupby('date')['sprtrn'].transform('mean')
    
#     # 计算每支股票的收益率
#     df['stock_returns'] = df.groupby('TICKER')['RET'].pct_change()
    
#     def rolling_regression(group):
#         X = sm.add_constant(group['market_returns'])
#         y = group['stock_returns']
        
#         beta = pd.Series(index=group.index, dtype=float)
#         idio_vol = pd.Series(index=group.index, dtype=float)
        
#         for i in range(window, len(group) + 1):
#             model = sm.OLS(y[i-window:i], X[i-window:i]).fit()
#             beta.iloc[i-1] = model.params['market_returns']
#             idio_vol.iloc[i-1] = np.std(model.resid)
        
#         return pd.DataFrame({'Beta': beta, 'Idio_vol': idio_vol})
    
#     result = df.groupby('TICKER').apply(rolling_regression)
#     result = result.reset_index()
#     result = result.rename(columns={'level_1': 'date'})
    
#     # 合并结果回原始数据框
#     df = pd.merge(df, result, on=['date', 'TICKER'], how='left')
    
#     # 填充缺失值
#     for indicator in ['Beta', 'Idio_vol']:
#         df[indicator] = df.groupby('TICKER')[indicator].transform(lambda x: x.fillna(x.median()))
#         df[indicator] = df[indicator].fillna(df[indicator].median())
    
#     # 处理极端值 (可选)
#     for indicator in ['Beta', 'Idio_vol']:
#         low = df[indicator].quantile(0.01)
#         high = df[indicator].quantile(0.99)
#         df[indicator] = df[indicator].clip(low, high)
    
#     # 计算 BetaSquared
#     df['BetaSquared'] = df['Beta'] ** 2
    
#     return df

# # 使用函数
# df = calculate_rolling_beta_and_idio_vol(df)

In [ ]:
def handle_missing_values_small_missing(df):
    # 处理分类变量
    category_vars = ['COMNAM', 'SHRCD', 'EXCHCD', 'SICCD', 'TICKER_encoded', 'SHRCD_encoded', 'EXCHCD_encoded', 'SICCD_encoded']
    df[category_vars] = df.groupby('TICKER')[category_vars].fillna(method='ffill')

    # 处理数值变量
    numeric_vars = ['PRC', 'VOL', 'SHROUT', 'BIDLO', 'ASKHI', 'OPENPRC', 'vwretd', 'ewretd', 'sprtrn', 'RET_calc', 'RET_interp']
    df[numeric_vars] = df.groupby('TICKER')[numeric_vars].transform(lambda x: x.interpolate(method='linear'))

    # 特殊处理 RET
    df['RET'] = df.groupby('TICKER')['RET'].fillna(0)  

    small_missing = ['ShortTermReversal', 'LME', 'r2_1', '52_week_high', 'LTurnover', 'Volatility']
    df[small_missing] = df.groupby('TICKER')[small_missing].fillna(method='ffill')


    # 重新计算 market_cap, ME, 和 Size
    df['market_cap'] = df['PRC'].abs() * df['SHROUT']
    df['ME'] = df['market_cap']
    df['Size'] = np.log(df['ME'])

    return df

# 应用函数
df = handle_missing_values_small_missing(df)

In [ ]:
# 计算缺失值统计，转换为 DataFrame
null_summary = pd.DataFrame(df.isnull().sum(), columns=['MissingValueCount'])

# 使用 option_context 临时改变显示设置并直接显示 DataFrame
with pd.option_context('display.max_columns', None,  'display.max_colwidth', 20):
    display(null_summary)

null_summary.to_csv("missing_value_summary.csv")


In [ ]:
df.reset_index(inplace = True)
df.set_index('date',inplace = True)

In [ ]:
df.index

In [ ]:
# df.to_csv("final_data_after_missing_value_handle_with_better_features.csv")

In [ ]:
def dropIndex(dataset):
    try:
        dataset = dataset.drop(columns = ['level_0', 'index'])
    except Exception as e:
                print(f"Issue : {e}")
    return dataset
df = dropIndex(df)

In [ ]:
# try:
#     data = data.drop(['level_0_stk','index_stk','3_month_high', 'Rel_to_3month_high', '1_month_high', 'Rel_to_1month_high', 'RET_calc', 'RET_interp'], axis=1)
# except KeyError as e:
#     print(f"Columns not found: {e}")
# df.to_csv("merged_data_with_features.csv")

In [ ]:
# df = pd.read_csv("merged_data_with_features_lag_1_month.csv")

In [ ]:
# df_columns = pd.DataFrame(df.columns)
# df_columns.to_csv('final_data_columns.csv')
# print('output df columns')

In [ ]:
# feature_eng_df = pd.DataFrame()
# feature_eng_df['TICKER'] = df['TICKER']
# feature_eng_df['date'] = df['date']
# feature_eng_df[existing_vars] = df[existing_vars]


In [ ]:
# # 对于月度特征，使用当前月的数据
# # 对于季度特征，使用4个月前的数据
# sto_data_selected['QuarterlyFeature_Lagged'] = sto_data_selected.groupby('PERMNO')['QuarterlyFeature'].shift(4)

# # 对于年度特征，使用6个月前的数据
# sto_data_selected['AnnualFeature_Lagged'] = sto_data_selected.groupby('PERMNO')['AnnualFeature'].shift(6)

### (Paused) Further Feature Engineering with Kronecker Product (Macro x others)

In [ ]:
print(fin_columns, sto_columns, macro_columns)

In [ ]:
# for macro_var in macro_data_scaled.columns:
#     for fin_var in fin_data.columns:
#         fin_data[f'{fin_var}_{macro_var}'] = fin_data[fin_var] * macro_data_scaled[macro_var]


### More Lags and Hyper-parameter Opitimization

In [ ]:
# # 创建滞后特征
# def create_lag_features(df, lags, target):
#     for lag in lags:
#         df[f'{target}_lag{lag}'] = df[target].shift(lag)
#     return df

# # 假设我们选择滞后1到3
# lags = [1, 2, 3, 4]
# model_data = create_lag_features(model_data, lags, 'RET')

### Lag All Features

## 3.3 Model Training

## Normalization(FR)

In [ ]:
data = pd.read_csv('final_data_after_missing_value_handle_with_better_features.csv')

In [ ]:
def dropIndex(dataset):
    try:
        dataset = dataset.drop(columns = ['index'])
    except Exception as e:
                print(f"Issue : {e}")
    return dataset
data = dropIndex(data)

In [ ]:
# 如果日期已经是索引，先重置索引
data.reset_index(drop=True, inplace=True)

data['date'] = pd.to_datetime(data['date'])

# 设置多级索引
data.set_index(['date'], inplace=True)

In [ ]:
# try:
#     data = data.drop(['level_0_stk','index_stk','3_month_high', 'Rel_to_3month_high', '1_month_high', 'Rel_to_1month_high', 'RET_calc', 'RET_interp'], axis=1)
# except KeyError as e:
#     print(f"Columns not found: {e}")


In [ ]:
data.columns

In [ ]:
data.index

In [ ]:
pd.DataFrame(data.dtypes).to_csv('check_data_info().csv')

In [ ]:
data.describe()

In [ ]:
# description = data.describe()
# description.to_csv('data_description.csv')


In [ ]:
# handle fin related data
# 处理无穷大和NaN值
data = data.replace([np.inf, -np.inf], np.nan)

# 创建一个新的DataFrame来存储归一化后的数据
normalized_data = pd.DataFrame(index=data.index)


# 1. MinMaxScaler用于大多数财务比率
minmax_cols = ['CAPEI', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil', 'pe_exi', 'pe_inc', 'dpr', 'npm', 
               'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe', 'roce', 'efftax']
minmax_scaler = MinMaxScaler()
for date in normalized_data.index.unique():
    normalized_data[minmax_cols] = minmax_scaler.fit_transform(data[minmax_cols].fillna(data[minmax_cols].mean()))

# 2. 对数转换后StandardScaler用于严重偏斜的正值数据
log_standard_cols = ['ps', 'pcf', 'debt_ebitda', 'lt_ppent', 'dltt_be', 'debt_assets', 'debt_capital', 'de_ratio']
log_standard_scaler = StandardScaler()
for col in log_standard_cols:
    for date in normalized_data.index.unique():
        # 将负值设为很小的正数
        temp_data = data[col].clip(lower=1e-8)
        normalized_data[col] = log_standard_scaler.fit_transform(np.log1p(temp_data).values.reshape(-1, 1))

# 3. Yeo-Johnson变换用于可能包含负值的财务指标
yeojohnson_cols = ['aftret_eq', 'aftret_invcapx', 'aftret_equity', 'pretret_noa', 'pretret_earnat', 'GProf']
pt_yeo = PowerTransformer(method='yeo-johnson')
for date in normalized_data.index.unique():
    normalized_data[yeojohnson_cols] = pt_yeo.fit_transform(data[yeojohnson_cols].fillna(data[yeojohnson_cols].mean()))

# 4. 对于0-1之间的比率数据，使用缩放而不是logit转换
ratio_cols = ['equity_invcap', 'debt_invcap', 'totdebt_invcap', 'capital_ratio']
ratio_scaler = MinMaxScaler(feature_range=(0.001, 0.999))  # 避免0和1
for date in normalized_data.index.unique():
    normalized_data[ratio_cols] = ratio_scaler.fit_transform(data[ratio_cols].fillna(data[ratio_cols].mean()))

# 5. 对于周转率和转换天数，使用Yeo-Johnson而不是Box-Cox
turn_cols = ['inv_turn', 'at_turn', 'rect_turn', 'pay_turn', 'cash_conversion']
pt_yeo_turn = PowerTransformer(method='yeo-johnson')
for date in normalized_data.index.unique():
    normalized_data[turn_cols] = pt_yeo_turn.fit_transform(data[turn_cols].fillna(data[turn_cols].mean()))

# 6. QuantileTransformer用于可能存在极端值的指标
quantile_cols = ['intcov', 'intcov_ratio', 'cash_ratio', 'quick_ratio', 'curr_ratio', 'sale_invcap', 'sale_equity', 'sale_nwc']
qt = QuantileTransformer(output_distribution='normal')
for date in normalized_data.index.unique():
    normalized_data[quantile_cols] = qt.fit_transform(data[quantile_cols].fillna(data[quantile_cols].mean()))

# 7. RobustScaler用于其他可能包含异常值的列
robust_cols = ['int_debt', 'int_totdebt', 'cash_lt', 'invt_act', 'rect_act', 'debt_at', 'short_debt', 'curr_debt', 'lt_debt', 
               'profit_lct', 'ocf_lct', 'cash_debt', 'fcf_ocf', 'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb', 'PEG_trailing', 'divyield',
              'r2_1', 'r12_2', 'r12_7', 'Volatility', 'SUV', '52_week_high', 'Rel_to_high']
robust_scaler = RobustScaler()
for date in normalized_data.index.unique():
    normalized_data[robust_cols] = robust_scaler.fit_transform(data[robust_cols].fillna(data[robust_cols].mean()))

# 检查是否所有列都已处理
unprocessed_cols = set(numeric_columns) - set(normalized_data.columns)
if unprocessed_cols:
    print(f"Warning: The following columns were not processed: {unprocessed_cols}")
    # 对于未处理的列，可以选择直接复制原始值或应用默认的StandardScaler
    for col in unprocessed_cols:
        for date in normalized_data.index.unique():
            normalized_data[col] = StandardScaler().fit_transform(data[[col]].fillna(data[col].mean()))



In [ ]:
# normalized_data.index

In [ ]:
# normalized_data.columns

In [ ]:
print(normalized_data.head())


In [ ]:
# handle stock and econ related data

# 创建归一化器
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# 选择需要进行 Min-Max 归一化的特征
min_max_features = [
    'vwretd', 'ewretd', 'sprtrn',
    'Beta', 'BetaSquared', 'idio_vol', 'spread', 'BM', 'ME', 'Size',
    'Momentum', 'ShortTermReversal', 'LME', 'LTurnover'
]

# 选择需要进行 Z-score 标准化的特征
z_score_features = [
    'PRC', 'VOL', 'SHROUT', 'BIDLO', 'ASKHI', 'OPENPRC',
    'market_cap', 'TB3MS', 'T10Y2Y', 'BAA10Y', 'DollarVolume'
]

# 进行 Min-Max 归一化
for date in normalized_data.index.unique():
    normalized_data[min_max_features] = min_max_scaler.fit_transform(data[min_max_features])

# 进行 Z-score 标准化
for date in normalized_data.index.unique():
    normalized_data[z_score_features] = standard_scaler.fit_transform(data[z_score_features])

# 显示归一化后的数据
print(normalized_data.head())


In [ ]:
normalized_data['TICKER'] = data['TICKER']
normalized_data['RET'] = data['RET']

In [ ]:
# normalized_data.to_csv('normalized_data.csv', index=True)

print("Normalization completed. Data saved to 'normalized_data.csv'")

Rank standardization in inreval [-1,1]
A rank standardization procedure following Kelly, Pruitt, and Su (2019) and Freyberger, Neuhierl, and Weber (2020) is employed. The stock characteristics are ranked month-by-month cross-sectionally and the ranks are mapped into the [-1,1] interval, thereby transforming features into a uniform distribution and increasing the insensitivity to outliers

In [ ]:
temp_nor_data = normalized_data.copy()

In [ ]:
all_features = (minmax_cols + log_standard_cols + yeojohnson_cols +
                ratio_cols + turn_cols + quantile_cols + robust_cols +
                min_max_features + z_score_features)

# 对每个月（由 'eom' 列指示的月份）进行分组，然后对这些特征进行百分比排名
temp_nor_data[all_features] = temp_nor_data[all_features].rank(pct=True)

# 将排名转换为 [-1, 1] 区间
temp_nor_data[all_features] = 2 * temp_nor_data[all_features] - 1

# 检查转换结果
print(temp_nor_data[all_features].head())


In [ ]:
temp_nor_data['TICKER'] = data['TICKER']
temp_nor_data['RET'] = data['RET']

In [ ]:
# 保存归一化后的数据
# temp_nor_data.to_csv('temp_nor_data.csv', index=True)

print("Normalization completed. Data saved to 'normalized_data.csv'")

In [ ]:
temp_nor_data.columns

In [ ]:
temp_nor_data.describe()

# Model Training

In [ ]:
#先用正常的经过二次feature过的data（final_data_after_missing_value_handle_with_better_features）试试
# model_data = pd.read_csv('final_data_after_missing_value_handle_with_better_features.csv')

#再用归一化的data（temp_nor_data）试一试
model_data = pd.read_csv('temp_nor_data.csv')

#再用标准化的data（normalized_data）试一试
# model_data = pd.read_csv('normalized_data.csv')


In [ ]:
model_data.columns

In [ ]:
# 如果日期已经是索引，先重置索引
model_data.reset_index(drop=False, inplace=True)

model_data['date'] = pd.to_datetime(model_data['date'])

# 设置多级索引
model_data.set_index(['date'], inplace=True)

# def dropIndex(dataset):
#     try:
#         dataset = dataset.drop(columns = ['index'])
#     except Exception as e:
#         print(f"Issue : {e}")
#     return dataset
# df = dropIndex(df)

In [ ]:
# 分割数据集为训练集和最后一个月的预测集
last_month = model_data.index.max()
model_data_for_training = model_data[model_data.index < last_month]
model_data_using_model_predicting = model_data[model_data.index == last_month]


In [ ]:
# 可能需要删除的变量
variables_to_drop = [
    'CAPEI', 'evm', 'pe_exi', 'pe_inc', 'ptpm', 'pcf',
    'aftret_eq', 'aftret_invcapx', 'aftret_equity',
    'pretret_noa', 'pretret_earnat',
    'equity_invcap', 'debt_invcap', 'totdebt_invcap',
    'intcov', 'intcov_ratio',
    'sale_invcap', 'sale_equity', 'sale_nwc',
    'int_debt', 'int_totdebt',
    'invt_act', 'rect_act',
    'short_debt', 'curr_debt', 'lt_debt',
    'ptb', 'PEG_trailing'
]

# existing_vars = [
    # 'market_cap',  # ME (Market Equity)
    # 'bm',          # Book-to-Market
    # 'pe_op_basic', # E2P (inverse)
    # 'pe_op_dil',   # Another version of E2P (inverse)
    # 'npm',         # Similar to PM (Profit Margin)
    # 'gpm',         # Similar to PCM (Price-to-Cost Margin)
    # 'roa',         # Return on Assets
    # 'roe',         # Return on Equity
    # 'ps',          # Price to Sales (inverse of S2P)
    # 'debt_assets', # Similar to Lev (Leverage)
    # 'at_turn',     # Similar to ATO (Asset Turnover)
    # 'accrual',     # Might correspond to OA (Operating Accruals)
    # 'divyield',    # Dividend Yield
# ]

# 需要保留的变量
variables_to_keep = [
    # 基准特征
    'Size', 'bm', 'Momentum',

    # Time-lag
    'r2_1', 'r12_2', 'r12_7', 
    # 'r36_13',  
    
    # 股票水平预测器
    'ShortTermReversal', 'market_cap', 'DollarVolume', 'spread',
    'RET', 'idio_vol', 'Beta', 'BetaSquared', 
    # 'ME',

    'pe_op_basic', 'pe_op_dil',
    
    # 波动性
    'Volatility', 'SUV',

    # 技术指标
    '52_week_high', 'Rel_to_high',
    
    # 宏观经济预测器 
    'TB3MS', 'T10Y2Y', 'BAA10Y', 
    
    # 其他重要财务指标
    'roa', 'roe', 'roce', 'debt_assets', 'debt_capital', 'de_ratio',
    'cash_ratio', 'quick_ratio', 'curr_ratio', 'at_turn', 'inv_turn',
    'accrual',
    
    # 价格和交易相关
    # 'PRC', 'VOL', 
    # 'SHROUT', 'BIDLO', 'ASKHI', 'OPENPRC',
    
    # 市场回报指标
    'vwretd', 'ewretd', 'sprtrn',
    
    # 其他可能有用的指标
    # 'dpr', 'npm', 'opmbd', 'opmad', 'gpm', 'cfm', 'efftax', 'ps',
    # 'debt_ebitda', 'lt_ppent', 'dltt_be', 'capital_ratio',
    # 'rect_turn', 'pay_turn', 'cash_conversion',
    # 'cash_lt', 'debt_at', 'profit_lct', 'ocf_lct',
    # 'cash_debt', 'fcf_ocf', 'rd_sale', 'adv_sale', 'staff_sale',
    # 'evm',
    # 'divyield',

    # 'aftret_eq', 'intcov', 'short_debt', 
    
    # 其他计算得到的指标
    'LME', 'LTurnover', 
     
    
    # 标识符
    'TICKER'
]

# 使用这些列表来选择或删除变量
model_data = model_data[variables_to_keep]
# 全部保留

# # 或者
# df_dropped = df.drop(columns=variables_to_drop)

# 打印列表长度，以检查
# print(f"Number of variables to drop: {len(variables_to_drop)}")
# print(f"Number of variables to keep: {len(variables_to_keep)}")

# try:
#     rf_data = rf_data.drop([''], axis=1)
# except KeyError as e:
#     print(f"Columns not found: {e}")

#因为结果不好所以不选择变量了

In [ ]:
# pd.DataFrame(model_data.columns).to_csv('model_data_columns.csv')

In [ ]:
pd.DataFrame(model_data.dtypes).to_csv('model_data_dtypess.csv')

In [ ]:
# columns_have_nan = ['ShortTermReversal','LME','LTurnover']
# try:
#     model_data[columns_have_nan] = model_data[columns_have_nan].fillna(model_data[columns_have_nan].mean())
# except KeyError as e:
#     print(f"Columns not found: {e}")


### lag过了！！！！！

In [ ]:
# 定义不需要滞后的变量
non_lag_variables = ['TICKER', 'RET']
variables_to_keep = model_data.columns
non_annual_features = [
    'Size', 'Momentum', 'r2_1', 'r12_2', 'r12_7',
    'ShortTermReversal', 'DollarVolume', 'spread', 'idio_vol',
    'Beta', 'BetaSquared', 'Volatility', 'SUV',
    '52_week_high', 'Rel_to_high', 'TB3MS', 'T10Y2Y', 'BAA10Y',
    'vwretd', 'ewretd', 'sprtrn', 'LME', 'LTurnover'
]

# 定义需要滞后的变量
lag_variables = [var for var in variables_to_keep if var not in non_lag_variables]

# 假设model_data是一个包含所有数据的DataFrame，并且已经按日期和TICKER排序
# 对需要滞后的变量进行滞后处理
for var in lag_variables:
    if var in non_annual_features:
        # 对non_annual_features进行1个月的滞后
        model_data[f'{var}_lag'] = model_data.groupby('TICKER')[var].shift(1)
    else:
        # 其他变量保持6个月的滞后
        model_data[f'{var}_lag'] = model_data.groupby('TICKER')[var].shift(6)

# 删除原始的非滞后变量（除了不需要滞后的变量）
model_data = model_data.drop(columns=lag_variables)

# 重命名滞后变量，去掉"_lag"后缀
model_data = model_data.rename(columns={f'{var}_lag': var for var in lag_variables})

# 删除包含NaN值的行（由于滞后处理产生的第一个时间点）
model_data = model_data.dropna()

# 最终的model_data将包含所有滞后处理后的变量和不需要滞后的变量

In [ ]:
model_data.columns

In [ ]:
model_data.describe()

In [ ]:
# model_data['RET'] = df.loc[df.index.get_level_values('date') == model_data.index, 'RET'].iloc[0]
model_data['RET'].head()

In [ ]:
# model_data = model_data.drop(columns = 'index')

In [ ]:
# 如果日期已经是索引，先重置索引
model_data.reset_index(inplace=True)

# 设置多级索引
# model_data.set_index(['date', 'TICKER'], inplace=True)
model_data.set_index(['date'], inplace=True)


# 对MultiIndex进行排序
model_data.sort_index(inplace=True)

In [ ]:
# model_data.to_csv('model_data.csv')